## Importamos las librerías 

In [ ]:
# Librerías Generales
import os, json, time
from datetime import datetime, timedelta
from yt_dlp import YoutubeDL
from pathlib import Path
import subprocess
from typing import List, Dict, Tuple

# 3. Análisis
import re
from sentence_transformers import SentenceTransformer, util
import numpy as np

## Configuración de directorios 

In [ ]:
# Dependiendo de si se ejecuta en un script o en un notebook
ROOT = Path(__file__).resolve().parent.parent if "__file__" in globals() else Path(os.getcwd()).parent

BASE = ROOT / "rtve_dataset"
DATA = ROOT / "data"
VIDEO_DIR = BASE / "videos"
SUB_DIR = BASE / "subtitles"
META_DIR = BASE / "metadata"

for folder in [VIDEO_DIR, SUB_DIR, META_DIR]:
    folder.mkdir(parents=True, exist_ok=True)

print(f"Directorios creados: {VIDEO_DIR}, {SUB_DIR}, {META_DIR}")

Directorios creados: rtve_dataset\videos, rtve_dataset\subtitles, rtve_dataset\metadata


# Funciones Auxiliares

In [ ]:
def parse_date(line):
    """
    Parse a date from a line of text. Assumes line is formatted as "date - URL".

    Args:
        line: A line of text containing a date.

    Returns:
        datetime: The parsed date.
    """
    return line.split(" - ")[0], line.split(" - ")[1]


def convert_vtt_to_srt(vtt_path: Path):
    """
    Convert a VTT file to SRT format using ffmpeg. Deletes the original VTT file after conversion.

    Args:
        vtt_path: Path to the VTT file.

    Returns:
        None
    """
    srt_path = vtt_path.with_suffix(".srt")
    subprocess.run(["ffmpeg", "-i", str(vtt_path), str(srt_path)], check=True)
    vtt_path.unlink()  # Delete the original VTT file
    return srt_path
    

## 1. Obtención de los enlaces de los Vídeos

In [4]:
with YoutubeDL({"extract_flat": True, "dump_single_json": True}) as ydl:
    info = ydl.extract_info("https://www.rtve.es/play/videos/telediario-2/", download=False)
    
entries = info.get("entries", [])
print(f"Total de vídeos encontrados: {len(entries)}")

[rtve.es:program] Extracting URL: https://www.rtve.es/play/videos/telediario-2/
[rtve.es:program] telediario-2: Downloading webpage
[rtve.es:program] 135930: Downloading page 1
[download] Downloading playlist: Telediario 2 - Programa informativo en RTVE Play
[rtve.es:program] 135930: Downloading page 2
[rtve.es:program] 135930: Downloading page 3
[rtve.es:program] 135930: Downloading page 4
[rtve.es:program] 135930: Downloading page 5
[rtve.es:program] 135930: Downloading page 6
[rtve.es:program] 135930: Downloading page 7
[rtve.es:program] 135930: Downloading page 8
[rtve.es:program] 135930: Downloading page 9
[rtve.es:program] 135930: Downloading page 10
[rtve.es:program] 135930: Downloading page 11
[rtve.es:program] 135930: Downloading page 12
[rtve.es:program] 135930: Downloading page 13
[rtve.es:program] 135930: Downloading page 14
[rtve.es:program] 135930: Downloading page 15
[rtve.es:program] 135930: Downloading page 16
[rtve.es:program] 135930: Downloading page 17
[rtve.es:prog

## 2. Descarga de los Vídeos

In [17]:
# First try for a given dar's programme, known the URL
def download_rtve_program(url: str, fecha: str, tag: str = "TD2", quality: str | int = "best", modalities: int = 7) -> dict:
    '''
    Download a single RTVE programme video + subtitles + metadata given its URL.

    Args:
        url (str): The URL of the RTVE programme.
        tag (str): A tag to use in the output filename.
        quality (str | int): Desired video quality, can be "best" or an integer (e.g., 720).
        modalities (int): Desired modality to download. Coded as a bitmask abc:
            - a: 1 for video, 0 for no video
            - b: 1 for audio, 0 for no audio
            - c: 1 for subtitles, 0 for no subtitles
            Thus, having 7 possible modalities (1-7), as 0 is not taken into account. The modalities are:
            1: Subtitles only
            2: Audio only
            3: Subtitles + Audio
            4: Video only
            5: Video + Subtitles
            6: Video + Audio
            7: Video + Audio + Subtitles
    '''
    out_template = f"{tag}_{fecha}"

    # Determine the format based on the quality and modalities
    if modalities <= 0 or modalities > 7:
        raise ValueError("Invalid modalities selected. Must be between 1 and 7.")
    write_subtitles = modalities & 1 != 0
    download_audio = modalities & 2 != 0
    download_video = modalities & 4 != 0

    if not (download_video or download_audio):  # Only subtitles
        opts = {
            "writesubtitles": True,
            "writeautomaticsub": True,
            "subtitleslangs": ["es", "es-ES", "spa"],
            "subtitlesformat": "srt",
            "outtmpl": {"subtitle": str(SUB_DIR / (out_template + ".%(ext)s"))},
            "skip_download": True,
        }

        with YoutubeDL(opts) as ydl:
            info = ydl.extract_info(url, download=True)

        # Since we skipped the download, no postprocessors are applied. Thus, we manually convert VTT to SRT
        convert_vtt_to_srt(Path(SUB_DIR / (out_template + ".es.vtt")))  
    
        return info
    elif download_video and download_audio:  # Video + Audio (Subtitles will be added if requested)
        if isinstance(quality, int):
            format = f"bv*[ext=mp4][height<={quality}]+ba*[ext=m4a]"
        else:
            format = "bv*[ext=mp4]+ba*[ext=m4a]/b[ext=mp4]/bv*+ba/b"
    elif download_video:  # Only Video
        format = f"bv*[ext=mp4][height<={quality}]/bv*/b" if isinstance(quality, int) else "bv*[ext=mp4]/b"
    elif download_audio:  # Only Audio
        format = "bestaudio[ext=m4a]/bestaudio"

    opts = {
        # Prefer mp4 video + m4a audio, merge if needed
        "format": format,
        
        # Output paths
        "outtmpl": {
            "default": str(VIDEO_DIR / (out_template + ".mp4")),
            "subtitle": str(SUB_DIR / (out_template + ".%(ext)s")),
            "infojson": str(META_DIR / (out_template + ".info.json")),
        },
        
        # Subtitles
        "writesubtitles": write_subtitles,
        "writeautomaticsub": write_subtitles,
        "subtitleslangs": ["es", "es-ES", "spa"],
        "subtitlesformat": "srt",
        
        # Metadata JSON
        "writeinfojson": True,
        
        # Post-processing
        "postprocessors": [],
        
        "ignoreerrors": True,
        "retries": 5,
        "noprogress": False
    }

    # Set the postprocessors based on the modalities used
    if write_subtitles:
        opts["postprocessors"].append({
            "key": "FFmpegSubtitlesConvertor",
            "format": "srt"
        })

    if download_video:  
        opts["postprocessors"].append({
            "key": "FFmpegVideoRemuxer", "preferedformat": "mp4"
        })
    
    with YoutubeDL(opts) as ydl:
        info = ydl.extract_info(url, download=True)
    
    return info

In [18]:
# line = "19/08/25 - https://www.rtve.es/play/videos/telediario-2/21-horas-19-08-25/16701428/"
line = "21/07/25 - https://www.rtve.es/play/videos/telediario-2/21-horas-21-07-25/16672839/"
fecha, url = parse_date(line)
fecha = datetime.strptime(fecha, "%d/%m/%y").strftime("%d_%m_%y")
info = download_rtve_program(url, fecha, tag="TD2", quality="best", modalities=1)

if info:
    print(json.dumps({k: info[k] for k in ["id", "title", "duration"] if k in info}, indent=2, ensure_ascii=False))

[rtve.es:alacarta] Extracting URL: https://www.rtve.es/play/videos/telediario-2/21-horas-21-07-25/16672839/
[rtve.es:alacarta] 16672839: Downloading JSON metadata
[rtve.es:alacarta] 16672839: Downloading url information
[rtve.es:alacarta] 16672839: Downloading m3u8 information
[rtve.es:alacarta] 16672839: Downloading MPD manifest
[rtve.es:alacarta] 16672839: Downloading url information
[rtve.es:alacarta] 16672839: Downloading m3u8 information
[rtve.es:alacarta] 16672839: Downloading MPD manifest
[rtve.es:alacarta] 16672839: Downloading subtitles info
[info] 16672839: Downloading subtitles: es


[info] 16672839: Downloading 1 format(s): hls-4401
[info] Writing video subtitles to: Telediario - 21 horas - 21⧸07⧸25 [16672839].es.vtt
[download] Destination: Telediario - 21 horas - 21⧸07⧸25 [16672839].es.vtt
[download] 100% of   71.33KiB in 00:00:00 at 643.33KiB/s 
[MoveFiles] Moving file "Telediario - 21 horas - 21⧸07⧸25 [16672839].es.vtt" to "rtve_dataset\subtitles\TD2_21_07_25.es.vtt"
{
  "id": "16672839",
  "title": "Telediario - 21 horas - 21/07/25",
  "duration": 2475.28
}


In [ ]:
tag = "TD2"
out_template = f"{tag}_%(title).120s_%(id)s"
quality = 480  # Example quality setting, can be an int or "best"
if isinstance(quality, int):
        format = f"bv*[ext=mp4][height<={quality}]+ba*[ext=m4a]"
elif quality == "best":
    format = "bv*[ext=mp4]+ba*[ext=m4a]/b[ext=mp4]/bv*+ba/b"

opts = {
    # Prefer mp4 video + m4a audio, merge if needed
    "format": format,
    
    # Output paths
    "outtmpl": {
        "default": str(VIDEO_DIR / (out_template + ".mp4")),
        "subtitle": str(SUB_DIR / (out_template + ".%(ext)s")),
        "infojson": str(META_DIR / (out_template + ".info.json")),
    },
    
    # Subtitles
    "writesubtitles": True,
    "writeautomaticsub": True,
    "subtitleslangs": ["es", "es-ES", "spa"],
    "subtitlesformat": "srt",
    "listsubs": True,
    "listformats": True,
    
    # # Metadata JSON
    # "writeinfojson": True,
    # "paths": {"home": str(META_DIR)},
    
    # Post-processing
    "postprocessors": [
        {"key": "FFmpegSubtitlesConvertor", "format": "srt"},
        {"key": "FFmpegVideoRemuxer", "preferedformat": "mp4"},
    ],
    
    "ignoreerrors": True,
    "retries": 5,
    "noprogress": False
}

with YoutubeDL(opts) as ydl:
    info = ydl.extract_info(url, download=True)

[rtve.es:alacarta] Extracting URL: https://www.rtve.es/play/videos/telediario-2/13-08-25/16696598/
[rtve.es:alacarta] 16696598: Downloading JSON metadata
[rtve.es:alacarta] 16696598: Downloading url information
[rtve.es:alacarta] 16696598: Downloading m3u8 information
[rtve.es:alacarta] 16696598: Downloading MPD manifest
[rtve.es:alacarta] 16696598: Downloading url information
[rtve.es:alacarta] 16696598: Downloading m3u8 information
[rtve.es:alacarta] 16696598: Downloading MPD manifest
[rtve.es:alacarta] 16696598: Downloading subtitles info
[info] 16696598: Downloading subtitles: es


[info] Available formats for 16696598:
ID                  EXT RESOLUTION FPS |   FILESIZE   TBR PROTO | VCODEC        VBR ACODEC      ABR ASR MORE INFO
----------------------------------------------------------------------------------------------------------------------------
dash-audio=191931-0 m4a audio only     | ~ 58.46MiB  192k dash  | audio only        mp4a.40.2  192k 48k DASH audio, m4a_dash
dash-audio=191931-1 m4a audio only     | ~ 58.46MiB  192k dash  | audio only        mp4a.40.2  192k 48k DASH audio, m4a_dash
2                   mp4 unknown        |                  http  | unknown           unknown
3                   mp4 unknown        |                  http  | unknown           unknown
dash-video=2751259  mp4 1280x720    25 | ~838.06MiB 2751k dash  | avc1.640029 2751k video only          DASH video, mp4_dash
hls-3120            mp4 1280x720    25 | ~950.38MiB 3120k m3u8  | avc1.640029 2751k mp4a.40.2  192k
dash-video=3987707  mp4 1920x1080   25 | ~  1.19GiB 3988k dash 

## 3. Extracción de la información (Subtítulos y Metadatos)

### 3.1. Obtención manual de los enlaces de cada informativo

Por ahora, dado un archivo JSON referente a la información del telediario 2 (tratado como una playlist)*, extraemos los enlaces de cada uno de los vídeos de interés (dependiendo de fechas)

\* Obtenido usando `yt-dlp` con el siguiente comando:
```bash
yt-dlp --flat-playlist -J "https://www.rtve.es/play/videos/telediario-2/" > td2.json
```
O bien ejecutando la celda inmediatamente posterior a esta.

In [ ]:
opts_metadata = {
    "extract_flat": True,
    "dump_single_json": True,
    "skip_download": True
}
URL = "https://www.rtve.es/play/videos/telediario-2/"
with YoutubeDL(opts_metadata) as ydl:
    info = ydl.extract_info(URL, download=False)

with open(DATA / "td2.json", "w", encoding="utf-8") as f:
    json.dump(info, f, indent=2, ensure_ascii=False)

[rtve.es:program] Extracting URL: https://www.rtve.es/play/videos/telediario-2/
[rtve.es:program] telediario-2: Downloading webpage
[rtve.es:program] 135930: Downloading page 1
[download] Downloading playlist: Telediario 2 - Programa informativo en RTVE Play
[rtve.es:program] 135930: Downloading page 2
[rtve.es:program] 135930: Downloading page 3
[rtve.es:program] 135930: Downloading page 4
[rtve.es:program] 135930: Downloading page 5
[rtve.es:program] 135930: Downloading page 6
[rtve.es:program] 135930: Downloading page 7
[rtve.es:program] 135930: Downloading page 8
[rtve.es:program] 135930: Downloading page 9
[rtve.es:program] 135930: Downloading page 10
[rtve.es:program] 135930: Downloading page 11
[rtve.es:program] 135930: Downloading page 12
[rtve.es:program] 135930: Downloading page 13
[rtve.es:program] 135930: Downloading page 14
[rtve.es:program] 135930: Downloading page 15
[rtve.es:program] 135930: Downloading page 16
[rtve.es:program] 135930: Downloading page 17
[rtve.es:prog

In [ ]:
# Rango de fechas (inclusivo)
fecha_inicio = datetime(2025, 1, 1)  # Ejemplo: 1 de enero de 2025
fecha_fin = datetime(2025, 8, 11) # Ejemplo: 11 de agosto de 2025
days_diff = (fecha_fin - fecha_inicio).days

with open(DATA / "td2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

entries = data.get("entries", [])
print(type(entries))

if not entries:
    print("No se encontraron entradas en el archivo JSON.")

fechas = []
for i in range(days_diff + 1):
    fechas.append((fecha_inicio + timedelta(days=i)).strftime("%d/%m/%y"))

def extract_urls_by_date(dates, entries):
    """
    Extract video URLs from entries based on the provided dates. Omits dates that do not match any entry title (this is useful to avoid days without news, like weekends).

    Args:
        dates: list of dates in the format "dd/mm/yy" to search for in the entry titles.
        entries: list of entries containing video information. Each entry corresponds to a news video and contains, at least, a "title" and a "url".

    Returns:
        dict: A dictionary mapping dates to their corresponding video URLs.
    """
    result = {}
    for date in dates:
        for entry in entries:
            if date in entry.get("title", ""):
                result[date] = entry.get("url")
                break
    return result

urls_by_date = extract_urls_by_date(fechas, entries)


# Read the current state of the TXT file and complete the URLs
with open(ROOT / "video_urls.txt", "r") as f:
    lines = [line.strip() for line in f if line.strip()]

for line in lines:
    date, url = parse_date(line)
    if date not in urls_by_date:
        urls_by_date[date] = url  # Add the URL if it doesn't exist

print(f"Total de URLs encontradas: {len(urls_by_date)}")
print(urls_by_date)

# Write the updated URLs back to the TXT file
with open(ROOT / "video_urls.txt", "w") as f:
    for date, url in urls_by_date.items():
        f.write(f"{date} - {url}\n")

<class 'list'>
Total de URLs encontradas: 163
{'01/01/25': 'https://www.rtve.es/play/videos/telediario-2/telediario-21-horas-01-01-25/16392347/', '02/01/25': 'https://www.rtve.es/play/videos/telediario-2/telediario-21-horas-02-01-25/16393520/', '03/01/25': 'https://www.rtve.es/play/videos/telediario-2/telediario-21-horas-03-01-25/16394680/', '06/01/25': 'https://www.rtve.es/play/videos/telediario-2/21-horas-06-01-25/16396450/', '07/01/25': 'https://www.rtve.es/play/videos/telediario-2/21-horas-07-01-25/16397981/', '08/01/25': 'https://www.rtve.es/play/videos/telediario-2/21-horas-08-01-25/16399579/', '09/01/25': 'https://www.rtve.es/play/videos/telediario-2/21-horas-09-01-25/16401192/', '10/01/25': 'https://www.rtve.es/play/videos/telediario-2/21-horas-10-01-25/16402918/', '13/01/25': 'https://www.rtve.es/play/videos/telediario-2/13-01-25/16405609/', '14/01/25': 'https://www.rtve.es/play/videos/telediario-2/14-01-25/16407222/', '15/01/25': 'https://www.rtve.es/play/videos/telediario-2/

### 3.2. Aplicación de Formato a los Subtítulos

In [ ]:
def extract_srt_text(srt_path: Path):
    """
    Extract and clean text from an SRT file. Exports the time stamps and corresponding text to a JSON

    Args:
        srt_path: Path to the SRT file.

    Returns:
        str: The cleaned text extracted from the SRT file.
    """
    with open(srt_path, "r", encoding="utf-8") as file:
        content = file.read()

    # # Enfoque 1
    # content = re.sub(r"\d+\n", "", content)  # elimina números de línea
    # content = re.sub(r"\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}\n", "", content)  # elimina marcas de tiempo
    # content = re.sub(r"\n+", "\n", content)  # elimina líneas vacías múltiples
    
    # return content.strip()

    # Enfoque 2
    pattern = re.compile(
        r"(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})\n(.*?)(?=\n\n|\Z)",
        re.DOTALL
    )  # Patrón para capturar bloques del tipo:
       # 00:00:01,000 --> 00:00:04,000 \n *Texto* \n\n o fin del archivo (cada bloque individual de un SRT)

    fragments = []
    for start, end, text in pattern.findall(content):
        clean_text = " ".join(line.strip() for line in text.splitlines() if line.strip())
        fragments.append({
            "start": start,
            "end": end,
            "text": clean_text
        })

    # Exportar a JSON
    json_path = srt_path.with_suffix(".json")
    with open(json_path, "w", encoding="utf-8") as json_file:
        json.dump(fragments, json_file, ensure_ascii=False, indent=2)

    # Concatenar todo el texto limpio
    full_text = " ".join(fragment["text"] for fragment in fragments)

    return full_text

In [19]:
# srt_file = SUB_DIR / "TD2_19_08_25.es.srt"
srt_file = SUB_DIR / "TD2_21_07_25.es.srt"

clean_text = extract_srt_text(srt_file)
print(clean_text[:500])  # Print the first 500 characters of the cleaned text

La Audiencia Nacional ha retirado el pasaporte a la expresidenta de Adif y el exdirector de Carreteras. Isabel Pardo de Vera y Javier Herrero han negado ante el juez haber participado en la adjudicación de obras públicas a cambio de mordidas para la trama Koldo García. Según el sumario del caso, Cristóbal Montoro aprobó una partida de 2.200 millones a las renovables en los presupuestos de 2012 después de que su despacho hiciera contratos con empresas del sector. El juez cree que el exministro de


In [ ]:
max_oraciones = 2
# Ventanas de 2 oraciones con solapamiento
solapamiento = 1

def segmentar_texto(texto: str, max_oraciones: int = 2, solapamiento: int = 1) -> List[str]:
    """
    Segmenta un texto en fragmentos basados en un número máximo de oraciones con solapamiento.
    Args:
        texto (str): El texto a segmentar.
        max_oraciones (int): Número máximo de oraciones por fragmento.
        solapamiento (int): Número de oraciones que se solapan entre fragmentos consecutivos.
    Returns:
        List[str]: Lista de fragmentos de texto segmentados.
    """
    oraciones = re.split(r'(?<=[\.\?\!])\s+', texto.strip())

    fragmentos, i = [], 0
    while i < len(oraciones):
        fragmento = " ".join(oraciones[i:i + max_oraciones]).strip()
        if fragmento:
            fragmentos.append(fragmento)
        i += max_oraciones - solapamiento

    return fragmentos

fragmentos = segmentar_texto(clean_text, max_oraciones, solapamiento)
print(f"Número de fragmentos creados: {len(fragmentos)}")
print(fragmentos[:3])  # Print the first 3 fragments for verification

Número de fragmentos creados: 366
['La Audiencia Nacional ha retirado el pasaporte a la expresidenta de Adif y el exdirector de Carreteras. Isabel Pardo de Vera y Javier Herrero han negado ante el juez haber participado en la adjudicación de obras públicas a cambio de mordidas para la trama Koldo García.', 'Isabel Pardo de Vera y Javier Herrero han negado ante el juez haber participado en la adjudicación de obras públicas a cambio de mordidas para la trama Koldo García. Según el sumario del caso, Cristóbal Montoro aprobó una partida de 2.200 millones a las renovables en los presupuestos de 2012 después de que su despacho hiciera contratos con empresas del sector.', 'Según el sumario del caso, Cristóbal Montoro aprobó una partida de 2.200 millones a las renovables en los presupuestos de 2012 después de que su despacho hiciera contratos con empresas del sector. El juez cree que el exministro de Hacienda tuvo un papel nuclear en la presunta trama que investiga.']


#### 3.2.2: Exportar fragmentos una vez pasados por la capa de embeddings

In [ ]:
# Cambiar una vez completemos una primera pasada para que los embeddings se guarden según el telediario al que pertenezcan
modelo = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
X = modelo.encode(fragmentos, batch_size=64, normalize_embeddings=True).astype("float32")
np.save("emb_frag.npy", X)

### 3.3 Obtención de Fragmentos de Interés

#### 3.3.1: Filtro Léxico - No aplica en la presente versión, ya que la capa de embeddings es más robusta

In [ ]:
LÉXICOS = [
    r'\balcohol(ismo|es|ico|ímetros?)?\b', r'\balcoholemia\b', r'\betílic\w*\b',
    r'\bdroga(s)?\b', r'\bestupefaciente(s)?\b', r'\bsustancias?\b',
    r'\bnarco(tráfico|s)?\b', r'\bcocaína\b', r'\bheroína\b', r'\b(hach[ií]s|marihuana|cannabis)\b',
    r'\bbenzodiacepinas?\b', r'\bopiáceos?\b', r'\bmetadona\b', r'\bketamina\b', r'\bmdma\b',
    r'\banfetamin\w*\b', r'\bpsicotrópicos?\b', r'\bbotellón(es)?\b',
    r'\bredad(a|as)\b', r'\bincautaci(ón|ones)\b', r'\bdecomis(o|os|a)\b',
    r'\btráfico de drogas\b', r'\bventa de drogas\b', r'\bconsumo de drogas\b',
    r'\bcontrol(es)? de alcoholemia\b'
]
PAT_LEX = re.compile("|".join(LÉXICOS), re.IGNORECASE)

def filtrar_lexico(fragmentos: List[str], patrón: re.Pattern) -> List[Tuple[int, str]]:
    """
    Filtra fragmentos de texto que contienen términos relacionados con drogas y alcohol basados en un patrón léxico.

    Args:
        fragmentos (List[str]): Lista de fragmentos de texto.
        patrón (re.Pattern): Patrón regex para identificar términos relacionados con drogas y alcohol.

    Returns:
        List[Tuple[int, str]]: Lista de tuplas con el índice del fragmento y el fragmento que contiene términos relacionados.
    """
    out = []
    for i, fragmento in enumerate(fragmentos):
        if patrón.search(fragmento):
            out.append((i, fragmento))
    return out

fragmentos_relevantes = filtrar_lexico(fragmentos, PAT_LEX)
print(f"Número de fragmentos relevantes encontrados: {len(fragmentos_relevantes)}")


Número de fragmentos relevantes encontrados: 2


In [ ]:
print("Fragmentos relevantes:")
for idx, frag in fragmentos_relevantes:
    print(f"Índice: {idx}, Fragmento: {frag}")

Fragmentos relevantes:
Índice: 289, Fragmento: Y a pesar de que este año otra noticia ha ensombrecido el festival. Una joven ha fallecido por colapso y se investiga si fue por consumo de drogas.
Índice: 290, Fragmento: Una joven ha fallecido por colapso y se investiga si fue por consumo de drogas. Tomorrowland continúa.


#### 3.3.2: Embeddings y Similaridad

In [27]:
def similitud_semantica(Q, X, top_k: int = 5, devolver_matrix: bool = False) -> dict:
    """
    Calcula la similitud semántica entre una consulta y un conjunto de fragmentos.

    Args:
        Q: Embeddings de las consultas.
        X: Embeddings de los fragmentos.
        top_k: Número de fragmentos más similares a devolver. Defaults to 5.
        devolver_matrix: Si es True, incluye la matriz S de similitudes en la salida. Defaults to False.
    """
    # Garantizar tipos y dimensiones
    Q, X = np.asarray(Q, dtype="float32"), np.asarray(X, dtype="float32")

    if Q.ndim != 2 or X.ndim != 2 or Q.shape[1] != X.shape[1]:
        raise ValueError("Las matrices de embeddings deben ser 2D y tener la misma dimensión de características.")
    
    K, D = Q.shape
    N, _ = X.shape
    top_k = max(1, min(top_k, N))

    # Asegurar contigüidad en memoria
    if not Q.flags["C_CONTIGUOUS"]:
        Q = np.ascontiguousarray(Q)
    if not X.flags["C_CONTIGUOUS"]:
        X = np.ascontiguousarray(X)

    # Calcular similitudes
    S = Q @ X.T  # Dot product ~ Similitud coseno, ya que embeddings normalizados

    # Obtener los índices de los top_k fragmentos más similares
    idx_top = np.argpartition(S, -top_k, axis=1)[:, -top_k:]

    # Ordenar
    scores_top = np.take_along_axis(S, idx_top, axis=1)
    order = np.argsort(-scores_top, axis=1)
    idx_top = np.take_along_axis(idx_top, order, axis=1)
    scores_top = np.take_along_axis(scores_top, order, axis=1)

    # Agregar por fragmento (mejor consulta y score máximo)
    mejor_score_por_frag = S.max(axis=0)
    mejor_query_por_frag = S.argmax(axis=0)

    resultado = {
        "indices_por_consulta": idx_top,  # (K, top_k)
        "scores_por_consulta": scores_top,  # (K, top_k)
        "mejor_score_por_fragmento": mejor_score_por_frag,  # (N,)
        "mejor_query_por_fragmento": mejor_query_por_frag  # (N,)
    }
    if devolver_matrix:
        resultado["matriz_similitudes"] = S  # (K, N)
    return resultado


#### 3.3.2.2: Cargar los fragmentos desde memoria

In [28]:
# Fragmentos
X = np.load("emb_frag.npy", "r")

modelo = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Consultas
CONSULTAS = [
    "consumo de drogas",
    "venta de drogas",
    "tráfico de estupefacientes",
    "control de alcoholemia",
    "abuso de alcohol",
    "incautación de sustancias",
    "redada policial por drogas",
    "adicciones a las drogas y tratamiento",
    "intoxicación etílica",
    "botellón y jóvenes",
    "drogas y menores de edad",
    "fumar cigarros",
    "consumo de tabaco"
]

Q = modelo.encode(CONSULTAS, normalize_embeddings=True).astype("float32")

resultados = similitud_semantica(Q, X, top_k=5, devolver_matrix=False)

indices_por_consulta = resultados["indices_por_consulta"]
scores_por_consulta = resultados["scores_por_consulta"]
mejor_score_por_fragmento = resultados["mejor_score_por_fragmento"]
mejor_query_por_fragmento = resultados["mejor_query_por_fragmento"]

for i, consulta in enumerate(CONSULTAS):
    print(f"\nConsulta: {consulta}")
    for j in range(indices_por_consulta.shape[1]):
        idx_frag = indices_por_consulta[i, j]
        score = scores_por_consulta[i, j]
        print(f"  Fragmento {j+1}: Índice {idx_frag}, Score {score:.4f}, Texto: {fragmentos[idx_frag]}")


Consulta: consumo de drogas
  Fragmento 1: Índice 289, Score 0.3852, Texto: Y a pesar de que este año otra noticia ha ensombrecido el festival. Una joven ha fallecido por colapso y se investiga si fue por consumo de drogas.
  Fragmento 2: Índice 290, Score 0.3550, Texto: Una joven ha fallecido por colapso y se investiga si fue por consumo de drogas. Tomorrowland continúa.
  Fragmento 3: Índice 54, Score 0.2911, Texto: Esos dos asuntos, el caso Koldo-Cerdán y el caso Montoro, son munición que intercambian PP y PSOE. Para los socialistas, las acusaciones sobre Montoro retratan las etapas de Aznar, de Rajoy y también la de Feijoo.
  Fragmento 4: Índice 53, Score 0.2702, Texto: Por este servicio, las empresas habrían pagado a la trama cerca de 4 millones y medio de euros. Esos dos asuntos, el caso Koldo-Cerdán y el caso Montoro, son munición que intercambian PP y PSOE.
  Fragmento 5: Índice 108, Score 0.2587, Texto: Para empeorar la situación, la principal planta desalinizadora de la Fran

#### 3.3.3: Calibración del Umbral de Similaridad - TBD

## 4. Aplicación a Vídeo (Fragmentos de Interés) - TBD